In [ ]:
# TODO: fix w/ new changes to methods in Contract dataclass

## Renewability

Renewability is the piece of the equation that takes **ACV** and converts to **ARR**. The premise is relatively simple, filter out any items in the contract that are considered non-renewable. This is easily done with a flag for renewability.

In [1]:
from arr import (
    Contract,
    ContractHeader,
    ContractLine,
    build_contracts_table,
    build_acv_table,
)
from datetime import date

ramp_w_onboarding = Contract(
    1,
    ContractHeader(36_000, date(2024, 1, 1), date(2026, 12, 31)),
    [
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
        ContractLine(6_000, date(2025, 1, 1), date(2025, 12, 31), 1, True),
        ContractLine(6_000, date(2025, 1, 1), date(2025, 12, 31), 2, True),
        ContractLine(8_000, date(2026, 1, 1), date(2026, 12, 31), 1, True),
        ContractLine(8_000, date(2026, 1, 1), date(2026, 12, 31), 2, True),
        ContractLine(6_000, date(2024, 1, 1), date(2024, 12, 31), 3, False),
    ],
)

df = build_contracts_table([ramp_w_onboarding])

print("Contract Table w/ all items")
display(df)

print("Contract Table w/ only renewable items.")
display(df[df["renewable"] == True])

ImportError: cannot import name 'build_contracts_table' from 'arr' (c:\Users\CStallings\OneDrive - Rockwell Automation, Inc\Documents\py_files\arr_data_professional\arr\__init__.py)

In [ ]:
acv_table = build_acv_table([ramp_w_onboarding])

arr_table = acv_table.loc[:, :, True]

arr_table.loc["total", :] = arr_table.sum().values
acv_table.loc["total", :] = acv_table.sum().values

print("ACV table")
display(acv_table)

print("ARR table")
display(arr_table)

ACV (including the `renewable == False`) starts at $\text{14k}$ in year 1, decrease to $\text{12k}$ in year 2, and up to $\text{16k}$ in year 3. While ARR (excluding the `renewable == False`) starts at $\text{8k}$ in year 1 and does a steady $\text{4k}$ increase each year.

$
\begin{matrix}
\text{Metric} & \text{Year 1} & \text{Year 2} & \text{Year 3}\\\\
\text{ACV} & \text{14k} & \text{12k} & \text{16k}\\
\text{ARR} & \text{8k} & \text{12k} & \text{16k}
\end{matrix}
$



### SCD Type 2

The decision to make a product renewable or non-renewable is a business decision. It is common for businesses to change their minds and adapt as new information comes, so the data will need to be able to adapt with it.

> Scenario:
> New leadership comes into the company at the start of 2024. The company is currently selling 4 different software items and 1 onboarding item:
> |Item Name|Item Sku|Renewable|
> |---|---|---|
> |Software Feature 1|1|True|
> |Software Feature 2|2|True|
> |Software Feature 3|3|True|
> |Software Feature 4|4|True|
> |Onboarding|5|False|
> 
> They have decided to update 'Software Feature 4' as no longer a renewable item:
> |Item Name|Item Sku|Renewable|
> |---|---|---|
> |Software Feature 1|1|True|
> |Software Feature 2|2|True|
> |Software Feature 3|3|True|
> |Software Feature 4|4|**False**|
> |Onboarding|5|False|
>
> This change is effective immediately on January 1st 2024.

This scenario is a classic use case of [SCD (Slowly Changing Dimensions)](https://en.wikipedia.org/wiki/Slowly_changing_dimension). This writeup will not be covering what SCD is or how to build it out in a data model. Plenty of articles and books exist covering this concept.

<div class="alert alert-block alert-info">
It is important to be able to supply the business with product/item data on both a Type 1 & Type 2 basis.
</div>


In [ ]:
contract_pre_change = Contract(
    1,
    ContractHeader(24_000, date(2023, 1, 1), date(2025, 12, 31)),
    [
        ContractLine(4_000, date(2023, 1, 1), date(2023, 12, 31), 4, True),
        ContractLine(4_000, date(2023, 1, 1), date(2023, 12, 31), 3, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 4, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 3, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 4, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 3, True),
    ],
)
contract_post_change = Contract(
    1,
    ContractHeader(24_000, date(2023, 1, 1), date(2025, 12, 31)),
    [
        ContractLine(4_000, date(2023, 1, 1), date(2023, 12, 31), 4, False),
        ContractLine(4_000, date(2023, 1, 1), date(2023, 12, 31), 3, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 4, False),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 3, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 4, False),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 3, True),
    ],
)

In [ ]:
pre_df = build_acv_table([contract_pre_change]).loc[:, :, True]
post_df = build_acv_table([contract_post_change]).loc[:, :, True]

type2_part1 = pre_df[[col for col in pre_df.columns if col < date(2024, 1, 31)]]
type2_part2 = post_df[[col for col in pre_df.columns if col >= date(2024, 1, 31)]]

type2 = type2_part1.merge(
    type2_part2, left_index=True, right_index=True, how="outer"
).fillna(0)

type1 = post_df

print("Type 2")
display(type2)

print("Type 1")
display(type1)

Depending on how the business asks for the data, would determine how to show it.

$
\begin{matrix}
\text{SCD} & \text{Year 1} & \text{Year 2} & \text{Year 3}\\\\
\text{Type 2} & \text{8k} & \text{4k} & \text{4k}\\
\text{Type 1} & \text{8k} & \text{8k} & \text{8k}
\end{matrix}
$

### Product Families

Grouping together and forming hierarchies in reporting is common place for businesses, skus are no exception. A common term to express this grouping is called a *Product Family*. Product families provide the ability to compare *apples to apples* data with many skus under similar feature sets. It also provides a simpler way to determine renewability for companies with rather large product catalogs.

<div class="alert alert-block alert-warning">
Imagine a company with over a million skus. They may not have the ability to determine renewability at that deep a grain. So a product family hierarchy naturally gets developed. This can cause some implications that you should be ready for. The further up the hierarchy you make a decision on renewability, the more likely you are to have items that are not actually renewable. A larger enterprise company may acquire another company. The aquired companies feature set could be bundled into a product family that is renewable. Which means <i>every</i> sku is renewable, even things like their onboarding.

I'm not saying this is right or wrong. I'm just saying you should know what is happening. Normally the impact is immaterial to the business, and if it becomes material, the business will find a way to determine renewability at a deeper grain.
</div>

In [ ]:
import pandas as pd

product_family_df = pd.DataFrame(
    {
        "item_sku": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "product_family": [1, 1, 1, 1, 2, 2, 2, 3, 3, 3],
        "renewable": [
            True, True, True, True, True, True, True, False, False, False
        ],
    }
)
product_family_df

## Deferred ARR

Up until now, we've only considered ARR that is *active*. Meaning, is the contract active in the period we are looking at. When a contract is booked before the start date, especially with new sales, the contract can be considered 'deferred' up until it is active.

If a contract is booked in June 2023, starts January 2024, and ends December 2024, then it is deferred the second half of 2023 and active for all of 2024.

In [ ]:
deferred_example = Contract(
    1,
    ContractHeader(
        10_000,
        start_date=date(2024, 1, 1),
        end_date=date(2024, 12, 31),
        booking_date=date(2023, 6, 30), #
    ),
    [
        ContractLine(5_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
        ContractLine(5_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
    ],
)

In [ ]:
build_acv_table([deferred_example], include_deferred=True)

# include_deferred defaults to True.
# Explicitly calling it out informational purposes.

ARR is shown in the above dataframe to start in June 2023 and end in December 2024.

<div class="alert alert-block alert-info">
It is common for people to refer to <b>ARR</b> implying both <b>active</b> and <b>deferred</b>. The business case for this is around new sales. Take the example above. Where would it be better to show the initial increase in 10k ARR? In June 2023 or in January 2024? Imagine if this was a portionally large deal for the company, and leadership is in their board meeting for June 2023.
<br>
<br>
<b>-- June 2023 Board Meeting</b><br>
"We just closed a really large deal this month!" - Leadership<br>
"That's great! I don't see the 10k in the ARR #s for June." - The Board<br>
"That's because it won't start until January 2024." - Leadership<br>
<br>
<br>
<b>-- January 2024 Board Meeting</b><br>
"Sales were awful. We didn't close anything?" - Leadership<br>
"How come your ARR jumped 10k this month?" - The Board<br>
"Oh, that's the deal we closed back in June 2023." - Leadership<br>
<br>
<br>
See the problem? It's not an easy story, unless you include deferred. <b>ARR really means 'Active ARR' and 'Deferred ARR'</b>
</div>